In the first exercise, you will design a small road network in SUMO according to specific requirements and then define a particular demand for the network. Using the SUMO config files, you will bring together the road network and traffic demand into a simulation. With the help of a Python script, you will start the simulation and extract macroscopic traffic information from it.

At the beginning, enter your name and student number in the next cell and execute it. Subsequently, your individual task will be generated.

The exercise is considered passed if at least 10 out of the 12 sub-tasks are successfully implemented.

In [1]:
from aufg import *

Name = 'Thanh Tung Nguyen'
Matrikelnummer = 146349

generate_aufgabe(Matrikelnummer)


    Your task in the first exercise will be to define a traffic network with 4 intersections/nodes (1),
    the edges between the nodes must have a length of 150 meters (2). All edges should be
    drivable in both directions, and have 3 lanes per direction (3). Within the
    traffic network, it is not possible to turn. This means that at the intersections it is not possible to turn
    left or right (4).

    The intersections are equipped with traffic lights for traffic control (5). Each edge must turn green once during the phase cycle
    and each phase is followed by a corresponding yellow phase and a red phase of at least 3
    seconds each (6). In total, each traffic light must have a cycle time of 90 seconds (7).

    Further define a traffic demand of a total of 100 vehicles. A vehicle should start every 2 seconds in the simulation (8). The vehicles should
    start alternately at the outer edges of the traffic network in a clockwise direction (9).

    Then enable the simula

In [2]:
from sumolib import checkBinary
import os
import traci
import sys

if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")


In [3]:
def run():
    """execute the TraCI control loop"""
    # Data structures to track waiting times and vehicle counts per edge
    total_waiting_time = {}
    vehicle_count = {}
    vehicle_current_edge = {}  # Track each vehicle's current edge

    # main loop. do something every simulation step until no more vehicles are
    # loaded or running
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        # Update vehicle positions and count entries per edge
        for veh_id in traci.vehicle.getIDList():
            current_edge = traci.vehicle.getRoadID(veh_id)
            previous_edge = vehicle_current_edge.get(veh_id, None) #return None if no edge found

            if current_edge != previous_edge:
                #Vehicle enter new edge
                if current_edge not in vehicle_count:
                    vehicle_count[current_edge] = 0
                vehicle_count[current_edge] += 1
                vehicle_current_edge[veh_id] = current_edge # set previous edge = current edge

        #Calculate waiting time for each edge
        for edge_id in traci.edge.getIDList():
            waiting_time = traci.edge.getWaitingTime(edge_id)
            if edge_id not in total_waiting_time:
                total_waiting_time[edge_id] = 0
            total_waiting_time[edge_id] += waiting_time

    # end simulation
    sys.stdout.flush()
    traci.close()

    # #Calculate avg waiting time on each edge
    average_waiting_time = {}
    print("Total waiting time per edge:")
    for edge, time in total_waiting_time.items():
        if edge in ['E0','E1', '-E2', 'E3']: #Only this edge have traffic flow on it
            print(f"{edge}: {time}s")
            count = vehicle_count[edge]
            if count > 0:
                average_waiting_time[edge] = total_waiting_time[edge] / count
            else:
                average_waiting_time[edge] = 0

    #Print result
    print("Average waiting time per edge:")
    for edge in average_waiting_time:
        print(f"{edge}: {round(average_waiting_time[edge],2)} s")

In [4]:
if __name__ == "__main__":
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "sumo_config.sumocfg"]) #Restart Kernel if you get an error here
    # Start the simulation with the network and route files

    run()

Total waiting time per edge:
-E2: 10271.0s
E0: 10617.0s
E1: 13354.0s
E3: 9639.0s
Average waiting time per edge:
-E2: 93.37 s
E0: 94.79 s
E1: 120.31 s
E3: 90.08 s
